[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/notebooks/blob/main/camenduru's_diffusers_to_sd.ipynb)

In [ ]:
!pip install -q torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 torchtext==0.14.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu116 -U
!pip -qq install huggingface-hub
!wget -q https://gist.github.com/jachiam/8a5c0b607e38fcc585168b90c686eb05/raw/2af0a9c0237ed98b863a75e1db21d7ed5541094f/convert_diffusers_to_sd.py
!git lfs install
from huggingface_hub import notebook_login
!git config --global credential.helper store
notebook_login()

In [ ]:
old_model_name = "adambrody" #@param {type:"string"}
new_model_name = "camenduru" #@param {type:"string"}
!git clone https://huggingface.co/camenduru/$old_model_name

old_model_name_ckpt = f"/content/{old_model_name}_ckpt"
!mkdir $old_model_name_ckpt

ckpt_file = f"{old_model_name_ckpt}/{old_model_name}.ckpt"
!python convert_diffusers_to_sd.py --model_path /content/$old_model_name --checkpoint_path $ckpt_file 

from slugify import slugify
from huggingface_hub import HfApi, HfFolder, CommitOperationAdd
from huggingface_hub import create_repo
from IPython.display import display_markdown
api = HfApi()
your_username = api.whoami()["name"]
repo_id = f"{your_username}/{slugify(new_model_name)}"
with open(HfFolder.path_token, 'r') as fin: hf_token = fin.read();

create_repo(repo_id,private=True, token=hf_token)
api.create_commit(repo_id=repo_id,operations=[],commit_message=f"{old_model_name} to {new_model_name}",token=hf_token)
api.upload_folder(folder_path=old_model_name_ckpt,path_in_repo="",repo_id=repo_id,token=hf_token)

display_markdown(f'https://huggingface.co/{repo_id}', raw=True)